In [1]:
## Standard

import os
import sys
sys.path.append('../RD-systems-and-test-benches')

import math
from datetime import datetime

## Specific

from tqdm.notebook import tqdm
import copy
from concurrent.futures import ProcessPoolExecutor
import networkx as nx
from itertools import product
from itertools import chain

## Data handling 

import pandas as pd
import numpy as np
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows

import sklearn.metrics
import scipy.constants as scc
import seaborn as sns
import scipy.integrate as integrate
import scipy.optimize as sco

## Plot

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scienceplots
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])


import plotly.graph_objects as go
import plotly.io as pio

from IPython.core.display import HTML

## Own modumes

import model as ty
import heat_transfer as bht
import proc as pr

import plot_functions_here as pfun
import general as gen

import thermal.SK_standard as nm
import utils.plot_functions as RD_pfun
import repo_config as rconfig

## Physics

import fluids as fds
import ht 

In [2]:
folder_path = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel'
geometry_path = os.path.join(folder_path, 'Panels\\Panel_V4.1I.xlsx')
hypotheses_path = folder_path + '\\' + 'Hypotheses\\Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'Steady-state conditions inputs\\100424-tests-conditions-GMI.xlsx'
exp_path = folder_path + '\\' + 'Experiments\\Experiments-GMI.xlsx'

# panelSpecs = pr.import_geometry(geometry_path)
# hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

# steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre en compte
# stepConditions = steadyStateConditions_df.to_dict(orient='records')

In [3]:
recap = pd.read_excel(r'G:\Drive partagés\Cercle Hard\R&D\Thermal performance tests PVT\Recap_RD_TUV_SSI.xlsx')
recap = recap.drop(index=0)
recap.dropna(how='all', inplace=True)

groups = {key: group for key, group in recap.groupby('Design')}

for key, group in groups.items():
    pr.convert_to_K(group)

In [4]:
triplets = [
            {'design_name' : 'V4.1NI',
             'test_type' : 'SK'},
            {'design_name' : 'V4.1NI',
                'test_type' : 'dark'},
            {'design_name' : 'V4.1I',
                'test_type' : 'SK'},
            {'design_name' : 'V4.1I',
                'test_type' : 'dark'}
                ]

for triplet in triplets:
             
    triplet['panelSpecs'] = pr.import_geometry(os.path.join(f'{folder_path}\\Panels', f'Panel_V4.1I.xlsx'))

    if triplet['design_name'] == 'V4.1NI':
        triplet['panelSpecs']['part2']['lambd_abs_ins'] = 0.
        triplet['panelSpecs']['part2']['lambd_tube_ins'] = 0.

        pr.update_panelSpecs_after_modif(triplet['panelSpecs'])
        
    triplet['hyp'] = pr.create_dict_from_excel(hypotheses_path,'Main')
    triplet['hyp']['method_h_top_g_exchanger'] = "forced_with_coeff_steps"
    triplet['steadyStateConditions_df'] = groups[triplet['design_name']].loc[groups[triplet['design_name']]['Type'] == triplet['test_type']].copy()
    pr.adjust_steadyStateConditions(triplet['steadyStateConditions_df'],triplet['hyp'])
    triplet['steadyStateConditions_df'].reset_index(drop=True,inplace=True)

In [5]:
# k_ = 1
# tripletk = triplets[k_]

# panelSpecs = copy.deepcopy(tripletk['panelSpecs'])
# hyp = copy.deepcopy(tripletk['hyp'])
# steadyStateConditions_df = tripletk['steadyStateConditions_df']

# # hyp['method_h_top_g_exchanger'] = 'forced_turbulent_with_coeff'

# # Call the simulation function
# df_res, list_res = ty.simu_steadyStateConditions(panelSpecs, hyp, steadyStateConditions_df)

In [6]:
# df_residuals = ty.recap_residuals_steadyStateConditions(panelSpecs, df_res, list_res)
# df_residuals

In [7]:
# df_ref = triplets[k_]['steadyStateConditions_df'].loc[(triplets[k_]['steadyStateConditions_df']['u'] > 2) & (triplets[k_]['steadyStateConditions_df']['u'] < 3)]

# plt.scatter(df_ref['T_m - T_amb'], df_ref['Qdot'], marker='+', c=df_ref['u'], cmap='viridis')
# plt.colorbar()

# plt.scatter(df_res.loc[(df_res['u'] > 2) & (df_res['u'] < 3)]['T_m - T_amb'], df_res.loc[(df_res['u'] > 2) & (df_res['u'] < 3)]['Qdot_tube_fluid'])

In [8]:
# triplets[2]['steadyStateConditions_df']

In [9]:
# steadyStateConditions = steadyStateConditions_df.to_dict(orient='records')

In [10]:
# steadyStateConditions_df = triplets[2]['steadyStateConditions_df']
# df_res, list_res = ty.simu_steadyStateConditions(triplets[2]['panelSpecs'],triplets[2]['hyp'],steadyStateConditions_df)
# df_res['Qdot_experimental'] = steadyStateConditions_df['Qdot']

# mae = sklearn.metrics.mean_absolute_error(df_res['Qdot_tube_fluid'],df_res['Qdot_experimental'])
# print(mae)
# df_res[['u', 'Qdot_experimental','Qdot_tube_fluid']]

In [11]:
# df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, steadyStateConditions[0], hyp)

### Implementation

In [12]:
mapping_parameters_bounds = {

    'V4.1NI' : {
        'panelSpecs': {
            'meander' : {
                    'lambd_air' : (0.05e-3,2.e-3),
            }
        }
    },

    'V4.1I' : {
        'panelSpecs': {
            'meander' : {
                    'lambd_air' : (0.05e-3,2.e-3),
            }
        }
    },

    'shared_designs' : {
        'panelSpecs': {
            'pv' : {
                    'tau_g' : (0.87, 0.93),
                    'alpha_g' : (0.03,0.09),
                    # 'alpha_PV' : (0.87, 0.93),
                    'eps_PV' : (0.85, 0.95),
                    'eps_g' : (0.85, 0.95),
            },
            'meander' : {
                    'l_c' : (0.1e-3,0.6e-3),
                    'eps_tube' : (0.01,0.20),
                    'eps_ins' : (0.25,0.60),
            },
        }
    },

    'hyp': {
        'coeff_h_top_forced_range0' : (0.5, 3.),
        'coeff_h_top_forced_range1' : (0.5, 3.),
        'coeff_h_top_forced_range2' : (0.5, 3.),
    },
}

# Function to recursively flatten the parameter bounds
def flatten_parameters_bounds(d, parent_keys=[]):
    items = []
    for k, v in d.items():
        if isinstance(v, dict):
            items.extend(flatten_parameters_bounds(v, parent_keys + [k]))
        else:
            items.append((parent_keys + [k], v))
    return items

# Flatten the parameter bounds and extract parameter paths and bounds
flattened_parameters = flatten_parameters_bounds(mapping_parameters_bounds)
parameter_paths = [param[0] for param in flattened_parameters]
bounds = [param[1] for param in flattened_parameters]

mapping_parameters = {tuple(flattened_parameters[j][0]): j for j in range(len(flattened_parameters))}

In [13]:
def flatten_dict(d, parent_key=()):
    """ Recursively flattens a nested dictionary. Keys become tuples of the path."""
    items = []
    for k, v in d.items():
        new_key = parent_key + (k,)
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [14]:
def update_specs(params, design, panelSpecs):
    for key in mapping_parameters_bounds[design]['panelSpecs']:
        if key in list(set(panelSpecs['decomp'].values())) + ['pv']:
            zones = list(panelSpecs['decomp'].keys()) + ['pv'] if key == 'pv' else [_ for _, value in panelSpecs['decomp'].items() if value == key]
            for z in zones:
                for subkey in mapping_parameters_bounds[design]['panelSpecs'][key]:
                    panelSpecs[z][subkey] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]

                    if subkey == 'eps_tube':
                        panelSpecs['part1']['eps_tube'] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]
                        panelSpecs['part3']['eps_tube'] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]
        else:
            panelSpecs[key] = params[mapping_parameters[(design, 'panelSpecs', key)]]

def update_panelSpecs(params, design_name, panelSpecs):
    update_specs(params, design_name, panelSpecs)
    update_specs(params, 'shared_designs', panelSpecs)
    pr.update_panelSpecs_after_modif(panelSpecs)

def update_hyp(params, hyp):

    for key in mapping_parameters_bounds['hyp']:
        hyp[key] = params[mapping_parameters[('hyp', key)]]

In [15]:
def simulation_function(params, triplet):

    panelSpecs = copy.deepcopy(triplet['panelSpecs'])
    hyp = copy.deepcopy(triplet['hyp'])
    steadyStateConditions_df = triplet['steadyStateConditions_df']

    update_panelSpecs(params, triplet['design_name'], panelSpecs)
    update_hyp(params, hyp)

    # Call the simulation function
    df_res, list_res = ty.simu_steadyStateConditions(panelSpecs, hyp, steadyStateConditions_df)
    return df_res


# Global counter for objective function calls
call_count = 0

# def callback_function(xk):
#     print(f"Iteration {len(iteration_history)}: params = {xk}")
#     iteration_history.append(xk.copy())

def objective_function(params, triplets):

    global call_count
    call_count += 1

    total_error = 0.0

    for triplet in triplets:

        df_res = simulation_function(params, triplet)

        # Compute the error
        error = np.sqrt(((df_res['Qdot_tube_fluid'] - triplet['steadyStateConditions_df']['Qdot']) ** 2).mean())
        total_error += error

    return total_error

def run_optimization(initial_guess, bounds, triplets):
    result = sco.minimize(
        objective_function,
        x0=initial_guess,
        args=(triplets,),  # Ensure you're passing the right triplet or list of triplets
        bounds=bounds,
        method='L-BFGS-B',
        options={'disp': True}
    )
    return result

In [16]:
params_init = np.array([9.97600358e-04, 5.31001503e-04, 9.0e-01, 7.38124530e-02,
        9.0e-01, 9.0e-01, 2.65959510e-04,
        8.45078850e-02, 3.68645143e-01, 1.15642655e+00, 1.63744321e+00,
        1.96089536e+00])

### Run optimization with sco.minimize, method 'L-BFGS-B'

In [ ]:
# Global optim (4 triplets : V4.1I et V4.1NI en SK et dark)
# Number of runs with different initial guesses
num_runs = 1

# Store the results
results = []

for run in tqdm(range(num_runs), desc="Optimizations"):
    # Random initial guess within bounds
    # initial_guess = [np.random.uniform(low=b[0], high=b[1]) for b in bounds]
    initial_guess = params_init

    # Perform the optimization with the triplet (use triplets[2] or all triplets depending on your intent)
    result = run_optimization(initial_guess, 
                              bounds, 
                              triplets, 
                              )  # This is passing one triplet

    # Store the result
    results.append(result)

### Differentiel evolution with intermediate solutions

In [21]:
# Define a callback function to store intermediate results
intermediate_solutions = []

def callback(xk, convergence):
    intermediate_solutions.append(xk)

result = sco.differential_evolution(
    objective_function,
    bounds=bounds,
    args=(triplets,),
    strategy='best1bin',
    maxiter=1000,
    popsize=15,
    tol=0.01,
    callback=callback,
    polish=False  # Set to False to prevent local search polishing
)

In [ ]:
result

In [ ]:
intermediate_solutions

In [23]:
params_minima = [{tuple(param[0]): result['x'][i] for i, param in enumerate(flattened_parameters)}]

In [ ]:
params_minima

In [ ]:
# params_manual = params_minima[0]
params_manual[ ('hyp', 'coeff_h_top_forced_range0')] = 1.
params_manual[ ('hyp', 'coeff_h_top_forced_range2')] = 2.5
params_manual

In [76]:
df_res_minima = []
mae_minima = []
rmse_minima = []

for params_minimum in params_minima:
    mae_minima.append([])
    rmse_minima.append([])

    df_concat = pd.DataFrame()
    for triplet in triplets:
        df_res = simulation_function([param for param in params_manual.values()], triplet)

        triplet['df_res'] = df_res

        df_res['Qdot_experimental'] = triplet['steadyStateConditions_df']['Qdot']
        df_concat = pd.concat([df_concat, df_res], axis=0)
        mae_minima[-1].append(sklearn.metrics.mean_absolute_error(df_res['Qdot_tube_fluid'], df_res['Qdot_experimental']))
        rmse_minima[-1].append(np.sqrt(((df_res['Qdot_tube_fluid'] - triplet['steadyStateConditions_df']['Qdot']) ** 2).mean()))
    df_res_minima.append(df_concat)

In [ ]:
k_ = 0

u_tuples = [(0.,1.), (1.,2.), (2.,3.)]

colors = ['green', 'blue', 'red']

for i, u_tuple in enumerate(u_tuples):
    u_min, u_max = u_tuple

    df_ref = triplets[k_]['steadyStateConditions_df'].loc[(triplets[k_]['steadyStateConditions_df']['u'] > u_min) & (triplets[k_]['steadyStateConditions_df']['u'] < u_max)]
    print(len(df_ref))

    plt.scatter(df_ref['T_m - T_amb'], df_ref['Qdot'], marker='+', color=colors[i])

    df_simu = triplets[k_]['df_res'].sort_values(by='T_m - T_amb')
    df_simu = df_simu.loc[(df_simu['u'] > u_min) & (df_simu['u'] < u_max)]

    plt.plot(df_simu['T_m - T_amb'], df_simu['Qdot_tube_fluid'], label = f'{u_min} < u < {u_max}', color=colors[i])

plt.legend()

In [29]:
a = df_res_minima[0][['Qdot_tube_fluid', 'Qdot_experimental']]

In [30]:
check = df_res_minima[0][['T_m - T_amb', 'u', 'Qdot_experimental', 'Qdot_tube_fluid']]

In [ ]:
mae_minima

In [ ]:
params_minima

In [ ]:
k = 0
plt.scatter(triplets[k]['steadyStateConditions_df']['T_m - T_amb'], triplets[k]['steadyStateConditions_df']['Qdot'], marker='+',
            # c=triplets[k]['steadyStateConditions_df']['u'], cmap='viridis',
            color = 'blue'
            
            )
plt.colorbar()
k = 1
plt.scatter(triplets[k]['steadyStateConditions_df']['T_m - T_amb'], triplets[k]['steadyStateConditions_df']['Qdot'], marker='+',
            # c=triplets[k]['steadyStateConditions_df']['u'], cmap='plasma',
            color='green',
            )
plt.legend()
plt.colorbar()

In [ ]:
df_res_minima[0][['u','Qdot_tube_fluid', 'Qdot_experimental']]

In [ ]:
1500*20/3600

In [ ]:
results

In [ ]:
call_count

In [16]:
params_minima = [{tuple(param[0]): result['x'][i] for i, param in enumerate(flattened_parameters)} for result in results]

df_res_minima = []
rmse_minima = []

for params_minimum in params_minima:
    rmse_minima.append([])
    df_concat = pd.DataFrame()
    for triplet in [triplets[2]]:
        df_res = simulation_function([param for param in params_minima[0].values()], triplet)
        df_res['Qdot_experimental'] = triplet['steadyStateConditions_df']['Qdot']
        df_concat = pd.concat([df_concat, df_res], axis=0)
        rmse_minima[-1].append(np.sqrt(((df_res['Qdot_tube_fluid'] - triplet['steadyStateConditions_df']['Qdot']) ** 2).mean()))
    df_res_minima.append(df_concat)

In [ ]:
# Objective function, bounds, and triplets would be defined here
# def objective_function(...):
#     pass

# Number of runs with different initial guesses
num_runs = 10

# Store the results
results = []

# Generate random initial guesses within the bounds
initial_guesses = [[np.random.uniform(low=b[0], high=b[1]) for b in bounds] for _ in range(num_runs)]

# Use ProcessPoolExecutor to run optimizations in parallel
with ProcessPoolExecutor() as executor:
    # Parallel execution of the run_optimization function
    futures = [executor.submit(run_optimization,
                               initial_guess,
                               bounds,
                               triplets) for initial_guess in initial_guesses]
    
    # Collect results as they complete
    for future in tqdm(futures, desc="Optimizations"):
        results.append(future.result())

# The results list now contains the result of each optimization

In [ ]:
params_minima = [{tuple(param[0]): result['x'][i] for i, param in enumerate(flattened_parameters)} for result in results]

In [ ]:
# Number of runs with different initial guesses
num_runs = 20

# Store the results
results_all = []

try:
    # Generate random initial guesses within the bounds
    for run in tqdm(range(num_runs), desc="Optimizations"):
        # Random initial guess within bounds
        initial_guess = [np.random.uniform(low=b[0], high=b[1]) for b in bounds]

        # Perform the optimization
        result = sco.minimize(
            objective_function,
            x0=initial_guess,
            args=([triplets[2]]),
            bounds=bounds,
            method='L-BFGS-B'
        )

        # Store the result
        results_all.append(result)

except KeyboardInterrupt:
    print("\nOptimization interrupted. Partial results saved.")

params_minima_all = [{tuple(param[0]): result['x'][i] for i, param in enumerate(flattened_parameters)} for result in results_all]

In [12]:
df_res_minima = []
rmse_minima = []

for params_minimum in params_minima:
    rmse_minima.append([])
    df_concat = pd.DataFrame()
    for triplet in [triplets[2]]:
        df_res = simulation_function([param for param in params_minima[0].values()], triplet)
        df_res['Qdot_experimental'] = triplet['steadyStateConditions_df']['Qdot']
        df_concat = pd.concat([df_concat, df_res], axis=0)
        rmse_minima[-1].append(np.sqrt(((df_res['Qdot_tube_fluid'] - triplet['steadyStateConditions_df']['Qdot']) ** 2).mean()))
    df_res_minima.append(df_concat)

In [ ]:
params_minima

In [ ]:
triplets[2]

In [19]:
update_panelSpecs([param for param in params_minima[0].values()], triplets[2]['design_name'], triplets[2]['panelSpecs'])

In [ ]:
triplets[2]['panelSpecs']

In [ ]:
df_res_minima[2][['Qdot_tube_fluid', 'Qdot_experimental']]

In [ ]:
rmse_minima

In [ ]:
params_minima

In [ ]:
df_res_minima = []
rmse_minima = []

rmse_minima.append([])
df_concat = pd.DataFrame()
for triplet in triplets:
    df_res = simulation_function([param for param in params_new[0].values()], triplet)
    df_res['Qdot_experimental'] = triplet['steadyStateConditions_df']['Qdot']
    df_concat = pd.concat([df_concat, df_res], axis=0)
    rmse_minima[-1].append(np.sqrt(((df_res['Qdot_tube_fluid'] - triplet['steadyStateConditions_df']['Qdot']) ** 2).mean()))
df_res_minima.append(df_concat)

In [ ]:
flattened_parameters[0]

In [ ]:
params_minima

In [91]:
params_new = [
            {
            ('V4.1NI', 'panelSpecs', 'meander', 'lambd_air') : triplets[1]['panelSpecs']['part2']['lambd_air'],
            ('V4.1I', 'panelSpecs', 'meander', 'lambd_air') : triplets[2]['panelSpecs']['part2']['lambd_air'],
            ('shared_designs', 'panelSpecs', 'pv', 'tau_g') : triplets[2]['panelSpecs']['pv']['tau_g'],
            ('shared_designs', 'panelSpecs', 'meander', 'eps_tube') : triplets[2]['panelSpecs']['part2']['eps_tube'],
            ('shared_designs', 'panelSpecs', 'meander', 'eps_ins') : triplets[2]['panelSpecs']['part2']['eps_ins'],
            ('hyp', 'coeff_h_top_forced_range0') : hyp['coeff_h_top_forced_range0'],
            ('hyp', 'coeff_h_top_forced_range1') : hyp['coeff_h_top_forced_range1'],
            ('hyp', 'coeff_h_top_forced_range2') : hyp['coeff_h_top_forced_range2']
            }
            ]

In [ ]:
params_new

In [ ]:
params_minima[0]

In [ ]:
[params for params in params_new[0].values()]

In [ ]:
rmse_minima

In [ ]:
df_res_minima[0][['Qdot_experimental', 'Qdot_tube_fluid']]

In [68]:
# flattened_triplets = flatten_dict(triplets)
# flattened_hyp = flatten_dict(hyp)

# # Combine the results into a single dictionary
# params_new = [flattened_triplets | flattened_hyp]

In [ ]:
flattened_parameters

In [ ]:
params_minima

In [54]:
ref = copy.deepcopy(triplets[1]['panelSpecs'])

In [55]:
update_panelSpecs([params[1] for params in params_minima[0]], triplets[1]['design_name'], triplets[1]['panelSpecs'])

In [ ]:
triplets[1]

In [ ]:
df_res_minima[0][['Qdot_experimental', 'Qdot_tube_fluid']]

In [ ]:
rmse_minima

In [ ]:
df_res[['Qdot_top_conv','Qdot_top_rad','Qdot_PV_sky','Qdot_absfin_back_conv','Qdot_absfin_back_rad','Qdot_tube_back_conv', 'Qdot_tube_back_rad']]

In [ ]:
df_res['T_absfin_mean']

In [ ]:
df_res[['T_fluid_mean','T_absfin_mean','T_ins_absfin_mean']] - 273.15

In [ ]:
for subkey in mapping_parameters_bounds['panelSpecs'][key]:

In [ ]:
panelSpecs_copy['part1']['alpha_g']

In [ ]:
params_minima

In [ ]:
df_res = simulation_function([param[1] for param in params_minima[0]], panelSpecs_copy, hyp_copy, steadyStateConditions_df,mapping_parameters_bounds)

In [ ]:
steadyStateConditions_df['model'] = df_res['Qdot_tube_fluid']
steadyStateConditions_df['relative_error'] = (((steadyStateConditions_df['Qdot'] - steadyStateConditions_df['model']) / steadyStateConditions_df['Qdot'])*100)
steadyStateConditions_df[['Qdot', 'model', 'T_m - T_amb', 'u', 'relative_error']]

In [ ]:
params_minima

In [ ]:
print(panelSpecs['part2']['tau_g'])
print(panelSpecs['part2']['alpha_g'])
print(panelSpecs['part2']['alpha_PV'])
print(panelSpecs['part2']['eps_PV'])
# print(panelSpecs['part2']['eps_g'])

In [ ]:
params_minima

In [ ]:
[param[1] for param in params_minima[0]]

In [ ]:
df_res['Qdot_tube_fluid']

In [11]:
flatten_params_minima = list(chain(*params_minima))

In [ ]:
for params in tqdm(params_minima, desc="Simulations"):
    df_res_minima.append( simulation_function(params, panelSpecs_copy, hyp_copy, steadyStateConditions_df) )
    rmse_minima.append( np.sqrt(((df_res['Qdot_tube_fluid'] - steadyStateConditions_df['Qdot']) ** 2).mean()) )

In [25]:
min_params = [ results[0]['x'][i] for i in range(len(results[0]['x']))]

In [27]:
df_res = simulation_function(min_params, panelSpecs_copy, hyp_copy, steadyStateConditions_df)

In [31]:
steadyStateConditions_df['model'] = df_res['Qdot_tube_fluid']

In [ ]:
steadyStateConditions_df

In [ ]:
df_res = simulation_function([[0.6]], panelSpecs, hyp, steadyStateConditions_df)

In [30]:
# Create the initial guess as the midpoint of the bounds
initial_guess = [
    (b[0] + b[1]) / 2 for b in bounds
]

# Prepare the templates (copies of your original dictionaries)
panelSpecs_template = panelSpecs.copy()
hyp_copy = hyp.copy()

In [28]:
initial_guess = [0.2]

In [31]:
# Perform the optimization using L-BFGS-B method
result = sco.minimize(
    objective_function,
    x0=initial_guess,
    args=(panelSpecs_copy, hyp_copy, steadyStateConditions_df, steadyStateConditions_df),
    bounds=bounds,
    method='L-BFGS-B'
)

# Extract the optimal parameters
optimal_params = result.x

In [31]:
# Define bounds as a list of tuples
bounds_list = bounds

result = sco.differential_evolution(
    objective_function,
    bounds=bounds_list,
    args=(panelSpecs_template, hyp_copy, steadyStateConditions_df, steadyStateConditions_df),
    strategy='best1bin',
    maxiter=1000,
    popsize=15,
    tol=0.01,
)

optimal_params = result.x

In [ ]:
optimal_params

In [11]:
df_res, list_res = simulation_function(params, panelSpecs, hyp, steadyStateConditions_df)

In [ ]:
df_res_list = []
mae_list = []
rmse_list = []

for i, params in tqdm(enumerate(params_list), total=len(params_list)):

    df_res = simulation_function(params, panelSpecs, hyp, steadyStateConditions_df)
    mae = np.abs(df_res['Qdot_tube_fluid'] - steadyStateConditions_df['Qdot']).mean()
    rmse = np.sqrt(((df_res['Qdot_tube_fluid'] - steadyStateConditions_df['Qdot']) ** 2).mean())

    df_res_list.append(df_res)
    mae_list.append(mae)
    rmse_list.append(rmse)

In [ ]:
min_index = rmse_list.index(min(rmse_list))
params_list[min_index]

In [33]:
params_list = np.array(list(product(np.arange(0.80,0.98,0.02),
                                        np.array([0.05]),
                                        np.array([0.95]),
                                        np.array([0.9]),
                                        np.array([0.9]),
                                        np.array([0.5e-3,1e-3,1.5e-3]),
                                        np.array([0.5e-3,1e-3,1.5e-3])
                                        )))

In [22]:
params_list = np.array(list(product(np.arange(0.1,0.98,0.02)
                                        ))).tolist()

In [ ]:
params_list

In [39]:
params = ranges_matrix.tolist()

In [ ]:
len(params)

In [16]:
steadyStateConditions_df['Qdot/AG'] = steadyStateConditions_df['Qdot'] / panelSpecs['AG']

In [17]:
dic_names = {'T_in' : 'T_fluid_in0',
             'T_out' : 'T_fluid_out',
             'Qdot_1_AG' : 'Qdot/AG',
             'T_amb' : 'T_amb',
             'u' : 'u'}

In [21]:
tab_mat,tab_res,X = nm.find_SK(steadyStateConditions_df, dic_names, columns_list=[1,1,1,1,0,0,1,0,1])

In [ ]:
nm.display_a_i(X)

In [ ]:
steadyStateConditions_df

In [23]:
steadyStateConditions_df[f'{dic_names["Qdot_1_AG"]}_model'] = tab_mat @ X[0]

In [25]:
steadyStateConditions_df['relative error'] = (steadyStateConditions_df[f'{dic_names["Qdot_1_AG"]}_model'] - steadyStateConditions_df[dic_names['Qdot_1_AG']]) / steadyStateConditions_df[dic_names['Qdot_1_AG']]

In [ ]:
u_range = np.arange(0,2.8,0.01)
h_top_forced_array = np.array([bht.h_top_forced(25+273.15, 35+273.15, u, 1.8) for u in u_range])
h_top_forced_turb_array = np.array([bht.h_top_forced_turbulent(25+273.15, 35+273.15, u, 1.8) for u in u_range])

In [ ]:
h_df = pd.DataFrame({'u' : u_range, 'h_top_forced' : h_top_forced_array, 'h_top_forced_turb' : h_top_forced_turb_array})
h_df['ratio'] = h_df['h_top_forced_turb'] / h_df['h_top_forced']
# sns.histplot(h_df['ratio'], bins = 20)

In [ ]:
# plt.plot(u_range,h_top_forced_array,label='laminar then turbulent')
# plt.plot(u_range,h_top_forced_turb_array,label='turbulent')
plt.plot(u_range,h_df['ratio'])

In [11]:
# df_one,res = ty.simu_one_steady_state_all_he(panelSpecs,sC,hyp)
# df_residuals = ty.recap_residuals(panelSpecs, df_one, res)

In [13]:
df_res['Qdot_exp'] = steadyStateConditions_df['Qdot']
df_res['error'] = df_res['Qdot_tube_fluid'] - df_res['Qdot_exp']

In [ ]:
df = df_res.copy()

df = df.loc[df['u'] == 2.7]

ht_signs = {'Qdot_sun_PV': 1, 'Qdot_sun_glass': 1, 'Qdot_top_conv': -1, 'Qdot_top_rad': -1, 'Qdot_PV_sky': -1, 'Qdot_absfin_back_conv': -1, 'Qdot_absfin_back_rad': -1, 'Qdot_Base_back': -1, 'Qdot_tube_back_conv': -1, 'Qdot_tube_back_rad': -1, 'Qdot_tube_sky': -1}
ht_labels = ['Qdot_sun_PV', 'Qdot_sun_glass', 'Qdot_top_conv', 'Qdot_top_rad', 'Qdot_PV_sky', 'Qdot_absfin_back_conv', 'Qdot_absfin_back_rad', 'Qdot_tube_back_conv', 'Qdot_tube_back_rad', 'Qdot_tube_sky']
colors_ht = [RD_config.get_hex_code('yellow',100), RD_config.get_hex_code('orange',100),RD_config.get_hex_code('green',150), RD_config.get_hex_code('blue',150),RD_config.get_hex_code('blue',100),RD_config.get_hex_code('green',100),RD_config.get_hex_code('grey',100),RD_config.get_hex_code('green',50),RD_config.get_hex_code('grey',50),RD_config.get_hex_code('blue',50)]

# Calculate the total positive heat transfers for each T_m - T_amb
df['positive_total'] = df[[label for label in ht_labels if ht_signs[label] == 1]].sum(axis=1)

# Initialize the figure
fig = go.Figure()

# Define the bar width
bar_width = 0.4

# Add traces for positive heat transfers (gains)
for i, label in enumerate(ht_labels):
    if ht_signs[label] == 1:
        fig.add_trace(go.Bar(
            x=df['T_m - T_amb'],
            y=df[label],
            name=label,
            width=[bar_width] * len(df),
            hovertext=round(df[label],0),
            hoverinfo='text',
            marker=dict(color=colors_ht[i])
        ))

# Add traces for negative heat transfers (losses) with base starting from the positive total
label_sum = []
for i, label in enumerate(ht_labels):
    if ht_signs[label] == -1:
        base = df['positive_total'] - df[[lbl for lbl in ht_labels if (ht_signs[lbl] == -1) & (lbl not in label_sum)]].sum(axis=1)
        fig.add_trace(go.Bar(
            x=df['T_m - T_amb'] + bar_width,  # Offset for side-by-side display
            y=df[label],
            name=label,
            width=[bar_width] * len(df),
            base=base,
            hovertext=round(df[label],0),
            hoverinfo='text',
            marker=dict(color=colors_ht[i])
        ))
        label_sum.append(label)


fig.add_trace(go.Scatter(
    x=df['T_m - T_amb'],
    y=df['Qdot_exp'],
    mode='lines',
    name='Qdot exp',
    line=dict(color='black', width=2)
))

fig.add_trace(go.Scatter(
    x=df['T_m - T_amb'],
    y=df['Qdot_tube_fluid'],
    mode='lines',
    name='Qdot exp',
    line=dict(color='grey', width=2)
))

# Update layout for stacked bars
fig.update_layout(barmode='relative', yaxis_title='Heat Transfer', xaxis_title='T_m - T_amb')


# Show the figure
fig.show()

In [ ]:
df['T_m - T_amb']

In [ ]:
df['negative_total']

In [ ]:
# df_res[['Qdot_tube_fluid','Qdot_exp','error']]

In [ ]:
mae = np.abs(df_res['error']).mean()
print(mae)

In [ ]:
wm = 7

# Deuxième écriture plus simple pour modifier les coefficients
mapping_matrix = np.array([
                            wm*[''],
                            ['tau_g', 'alpha_g', 'alpha_PV', 'eps_PV', 'eps_g', 'l_c', 'lambd_air'],
                           ])

part_mapping_matrix = [
                        np.nan,
                        [['part1','part2','part3'] for _ in range(wm)]
                        ]

ranges_matrix = np.array(list(product(np.arange(0.80,0.98,0.02),
                                        np.array([0.05]),
                                        np.array([0.95]),
                                        np.array([0.9]),
                                        np.array([0.9]),
                                        np.array([0.5e-3,1e-3,1.5e-3]),
                                        np.array([0.5e-3,1e-3,1.5e-3])
                                        )))

In [ ]:
panelSpecs['decomp']

In [ ]:
panelSpecs['part1']

In [24]:
wm = 1

# Deuxième écriture plus simple pour modifier les coefficients
mapping_matrix = np.array([
                            wm*[''],
                            ['lambd_air'],
                           ])

part_mapping_matrix = [
                        np.nan,
                        [['part1','part2','part3'] for _ in range(wm)]
                        ]

ranges_matrix = np.array(list(product(
                                        np.array([0.1e-3,10e-3,1.1e-4])
                                        )))

In [ ]:
ranges_matrix

In [ ]:
part_mapping_matrix[1]

In [ ]:
panelSpecs.keys()

In [ ]:
flattened_parameters

In [ ]:
df_res_list[9][['T_amb','u','T_fluid_in','Qdot_tube_fluid','Qdot exp','error']]

In [ ]:
min_index = mae_list.index(min(mae_list))
print(min_index)

In [ ]:
min(mae_list)

In [ ]:
df_res_list[3][['Qdot_tube_fluid','Qdot exp','error']]

In [ ]:
mae_list

In [ ]:
plt.scatter(np.arange(0,len(rmse_list),1),rmse_list)

In [ ]:
panelSpecs['pv']['tau_g']

In [ ]:
len(ranges_matrix)*2/3600

In [ ]:
all_combinations = list(product(range1, range2, range3))
matrix = np.array(all_combinations)

In [ ]:
matrix

Calibrage modèle multi-design

In [ ]:
# Permet de modifier une valeur du dictionnaire hyp
## Par exemple init_hyp('method_h_back_abs','free') va retourner le dictionnaire hyp avec la valeur 'method_h_back_abs' modifié en 'free'
def init_hyp(coeff, method) : 
    hyp[coeff] = method
    return(hyp)

Création de la matrice du modèle

In [ ]:
# Récupération des résultats expérimentaux

X_list_df = pd.read_excel(exp_path,header = 2) ## On ne prend pas en compe les 2 premières lignes // EN FONCTION DE COMMENT SONT PR2SENT2S LES RES
X_list = X_list_df.to_dict(orient='records')

In [ ]:
# Création de la matrice "A" sous forme de dictionnaire

measured_case = 0 # nb_measured_case = len(X_tot)
case = 0 # nb_case = len(stepConditions) il faudrait faire correspondre les lignes de stepConditions avec le numéro des measuredcase comme ça on a qu'une variable

# Dictionnaire des paramètres du modèle commun à tous les designs "alpha_0"
basic_parameters = dict([
    ('basic_parameters_1',1),
    ('basic_parameters_2',1),
    ('basic_parameters_3',1),
    ('basic_parameters_4',1),
    ('basic_parameters_5',1),
    ('basic_parameters_6',1),
    ('basic_parameters_7',1),
    ('basic_parameters_8',1),
    ('basic_parameters_9',1),
    ('basic_parameters_10',1),
    ])

# Matrice du modèle "A" 
model_matrix = dict(['alpha_0',basic_parameters])

# Plutôt faire l'inverse et créer le dictionnaire à partir d'une DataFrame puis sortir les différents X
# for i in range(nb_measured_case) :
# Point de données
X = dict([('componentSpecs',componentSpecs), ('stepConditions',stepConditions[case])]) 
# X_tot = dict([ 
#    ('X_{measured_case}',X)
#   ])

# Dictionnaire des paramètres du modèle liés à la géométrie "alpha_j"
design_parameters = dict([
    ('tau_g_{measured_case}', X['componentSpecs']['pv']['tau_g']), # Transmittance glass
    ('alpha_g_{measured_case}', X['componentSpecs']['pv']['alpha_g']), # Absorptance glass
    ('eps_PV_{measured_case}', X['componentSpecs']['pv']['eps_PV']), # Émissivité cellule PV
    ('eps_g_{measured_case}', X['componentSpecs']['pv']['eps_g']), # Émissivité glass
    ('eps_hx_top_{measured_case}', X['componentSpecs']['main']['eps_hx_top']), # Émissivité de l'échangeur thermique sur le panneau
    ('eps_hx_back_{measured_case}', X['componentSpecs']['main']['eps_hx_back']), # Émissivité de l'échangeur thermique à l’arrière du panneau
    ('eps_ins_{measured_case}', X['componentSpecs']['main']['eps_ins']), # Émissivité isolant
    ('lambd_air_{measured_case}',  X['componentSpecs']['main']['lambd_air']) # Épaisseur de la couche d'air
    ])

model_matrix['alpha_{measured_case}'] = design_parameters
#fin boucle for

In [ ]:
# Création de la matrice "A" sous forme de listes

measured_case = 0 # nb_measured_case = len(X_list)
case = 0 # nb_case = len(stepConditions) il faudrait faire correspondre les lignes de stepConditions avec le numéro des measuredcase comme ça on a qu'une variable

# Matrice du modèle "A" 
model_matrix = np.ones(10)

# Plutôt faire l'inverse et créer le dictionnaire à partir d'une DataFrame puis sortir les différents X
# for i in range(nb_measured_case) :
# Point de données
X = dict([('componentSpecs',componentSpecs), ('stepConditions',stepConditions[case])]) 
# X_list = dict([ 
#    ('X_{measured_case}',X)
#   ])

# Dictionnaire des paramètres du modèle liés à la géométrie "alpha_j"
design_parameters = np.zeros(10)
design_parameters[0] = X['componentSpecs']['pv']['tau_g'] # Transmittance glass
design_parameters[1] =  X['componentSpecs']['pv']['alpha_g'] # Absorptance glass
design_parameters[2] = X['componentSpecs']['pv']['eps_PV'] # Émissivité cellule PV
design_parameters[3] = X['componentSpecs']['pv']['eps_g'] # Émissivité glass
design_parameters[4] = X['componentSpecs']['main']['eps_hx_top'] # Émissivité de l'échangeur thermique sur le panneau
design_parameters[5] = X['componentSpecs']['main']['eps_hx_back'] # Émissivité de l'échangeur thermique à l’arrière du panneau
design_parameters[6] = X['componentSpecs']['main']['eps_ins'] # Émissivité isolant
design_parameters[7] = X['componentSpecs']['main']['lambd_air'] # Épaisseur de la couche d'air


model_matrix= np.vstack((model_matrix, design_parameters))

#fin boucle for
model_matrix

In [ ]:
m = len(correspondance_matrice)
nb_measured_case = len(X_list)

# MAJ de hyp en fonction des méthodes que l'on veut
hyp = init_hyp("method_h_back_abs", "free")


def Qdot_model_tube_fluid(X_list, model_matrix, correspondance_matrice) :
        Qdot_list = []
        
        # Actualise le modèle
        for i in range(m) : 
                if correspondance_matrice[i, 0] != 0. : 
                       hyp[correspondance_matrice[i,0]] = model_matrix[i, 0]

        # Actualise la géométrie
        for j in range(nb_measured_case) : 
                for i in range(m) :
                        if correspondance_matrice[i, 0] != 0. :
                                X_list[j]['componentSpecs'][correspondance_matrice[i,1]] = model_matrix[i, j]
                df_one = ty.simu_one_steady_state_all_he(X_list[j]['componentSpecs'], X_list[j]['stepConditions'], hyp)[0]
                Qdot_list.append(df_one['Qdot_tube_fluid'])
        return(Qdot_list)